# Proyecto integrador de filtros digitales
Este notebook implementa el análisis comparativo de filtros FIR, IIR y Notch aplicado a señales reales: audio, ECG y sensores inerciales.
Se estructura de acuerdo con los pasos de la guía práctica.

In [ ]:
# filtros.py mejorado
import numpy as np
from scipy.signal import firwin, remez, butter, filtfilt, iirnotch

def filtro_fir_ventana(signal, fs, orden, fc, ventana='hamming'):
    """Filtrado FIR mediante el método de ventanas."""
    coeff = firwin(orden + 1, fc, fs=fs, window=ventana)
    return filtfilt(coeff, [1], signal)

def filtro_fir_remez(signal, fs, orden, bandas, pesos):
    """Filtrado FIR óptimo usando el algoritmo de Parks-McClellan (remez)."""
    coeff = remez(orden + 1, bandas, pesos, fs=fs)
    return filtfilt(coeff, [1], signal)

def filtro_iir(signal, fs, orden, fc, btype='low'):
    """Filtrado IIR tipo Butterworth."""
    b, a = butter(orden, fc, btype=btype, fs=fs)
    return filtfilt(b, a, signal)

def filtro_notch(signal, fs, f0=50.0, Q=30.0):
    """Filtro notch para eliminar una frecuencia específica (como 50 o 60 Hz)."""
    b, a = iirnotch(f0 / (fs / 2), Q)
    return filtfilt(b, a, signal)

def filtro_lms(signal, desired, mu=0.01, orden=32):
    """Filtro adaptativo LMS simplificado."""
    w = np.zeros(orden)
    out = np.zeros_like(signal)
    for n in range(orden, len(signal)):
        x = signal[n - orden:n]
        y = np.dot(w, x)
        e = desired[n] - y
        w += 2 * mu * e * x
        out[n] = y
    return out


## Carga y normalización de señales

In [ ]:
# Audio
from scipy.io import wavfile
fs_audio, audio = wavfile.read('grabacion_ruido.wav')
audio = audio.astype(np.float32) / np.max(np.abs(audio))

# ECG
import pandas as pd
df_ecg = pd.read_csv('ecg_ruido.csv')
time_ecg = df_ecg['time'].values
ecg = df_ecg['ecg'].values.astype(np.float32)
fs_ecg = 1.0 / (time_ecg[1] - time_ecg[0])
ecg /= np.max(np.abs(ecg))

# Sensores
df_sens = pd.read_csv('sensor_vibracion.csv')
time_sens = df_sens['time'].values
fs_sens = 1.0 / (time_sens[1] - time_sens[0])
acc = df_sens[['acc_x', 'acc_y', 'acc_z']].values.astype(np.float32)
acc /= np.max(np.abs(acc), axis=0)

## Aplicación de filtros y análisis en señal de audio

In [ ]:
from filtros import filtro_fir_ventana, filtro_fir_remez, filtro_iir
orden_fir, fc_audio = 100, 3000
audio_fir = filtro_fir_ventana(audio, fs_audio, orden_fir, fc_audio)
audio_remez = filtro_fir_remez(audio, fs_audio, orden_fir, [0, 2800, 3200, fs_audio/2], [1, 1])
audio_iir = filtro_iir(audio, fs_audio, 4, fc_audio, btype='low')

In [ ]:
import timeit, numpy as np
def calcula_metricas(orig, filt):
    t0 = timeit.default_timer()
    _ = filtfilt([1], [1], orig)
    lat = timeit.default_timer() - t0
    ps = np.mean(orig**2)
    pn = np.mean((orig - filt)**2)
    snr_db = 10 * np.log10(ps / pn)
    return snr_db, lat

import pandas as pd
resultados_audio = []
for nombre, señal in [('FIR ventana', audio_fir), ('FIR Remez', audio_remez), ('IIR Butter', audio_iir)]:
    snr, lat = calcula_metricas(audio, señal)
    resultados_audio.append({'Método': nombre, 'SNR (dB)': snr, 'Latencia (s)': lat})
df_audio = pd.DataFrame(resultados_audio)
df_audio

## Aplicación de filtros y análisis en señal de ECG

In [ ]:
from filtros import filtro_notch
ecg_notch = filtro_notch(ecg, fs_ecg, f0=50.0, Q=30.0)
b_bp, a_bp = butter(4, [0.5/(fs_ecg/2), 40/(fs_ecg/2)], btype='band', fs=fs_ecg)
from scipy.signal import filtfilt
ecg_filt = filtfilt(b_bp, a_bp, ecg_notch)
snr_ecg, lat_ecg = calcula_metricas(ecg, ecg_filt)

In [ ]:
print(f"SNR ECG filtrado: {snr_ecg:.2f} dB — Latencia: {lat_ecg:.4f} s")
import matplotlib.pyplot as plt
plt.figure(figsize=(8,3))
plt.plot(time_ecg, ecg, label='Original', alpha=0.6)
plt.plot(time_ecg, ecg_filt, label='Notch + BP')
plt.legend(); plt.title('ECG: antes y después del filtrado')

plt.figure(figsize=(8,3))
plt.magnitude_spectrum(ecg, Fs=fs_ecg, scale='dB', color='C0', label='Original')
plt.magnitude_spectrum(ecg_filt, Fs=fs_ecg, scale='dB', color='C1', label='Filtrada')
plt.legend(); plt.title('Espectro ECG (dB)')

## Aplicación de filtros y análisis en sensores inerciales

In [ ]:
b_sens, a_sens = butter(3, [5/(fs_sens/2), 50/(fs_sens/2)], btype='band', fs=fs_sens)
acc_filt = filtfilt(b_sens, a_sens, acc, axis=0)
resultados_sens = []
for i, eje in enumerate(['X', 'Y', 'Z']):
    orig = acc[:, i]
    filt = acc_filt[:, i]
    snr, lat = calcula_metricas(orig, filt)
    resultados_sens.append({'Eje': eje, 'SNR (dB)': snr, 'Latencia (s)': lat})
df_sens = pd.DataFrame(resultados_sens)
df_sens

In [ ]:
plt.figure(figsize=(8,3))
plt.plot(time_sens, acc[:,0], label='Original X', alpha=0.6)
plt.plot(time_sens, acc_filt[:,0], label='Filtrado X')
plt.legend(); plt.title('Acelerómetro eje X: antes y después')